In [12]:
from pysat.solvers import Minisat22
from itertools import combinations

def optimize_packing(vehicle_dims, items):
    L, W, H = vehicle_dims
    num_items = len(items)
    max_v = 3  # On limite à 3 véhicules

    # Fonction pour encoder les variables : objet i placé au coin (x, y, z) dans le véhicule v
    def var_index(i, v, x, y, z):
        return 1 + i * (max_v * L * W * H) + v * (L * W * H) + x * (W * H) + y * H + z

    solver = Minisat22()

    # Contrainte 1 : Chaque objet doit être placé dans un véhicule avec une position valide
    for i, (L2, W2, H2, _) in enumerate(items):
        clause = []
        for v in range(max_v):
            for x in range(L - L2 + 1):  # Assure que l'objet reste dans les limites
                for y in range(W - W2 + 1):
                    for z in range(H - H2 + 1):
                        clause.append(var_index(i, v, x, y, z))
        solver.add_clause(clause)

    # Contrainte 2 : Les objets ne se chevauchent pas
    for (i1, (L1, W1, H1, _)), (i2, (L2, W2, H2, _)) in combinations(enumerate(items), 2):
        for v in range(max_v):  # Les objets ne se chevauchent que dans le même véhicule
            for x1 in range(L - L1 + 1):
                for y1 in range(W - W1 + 1):
                    for z1 in range(H - H1 + 1):
                        for x2 in range(x1, x1 + L1):
                            for y2 in range(y1, y1 + W1):
                                for z2 in range(z1, z1 + H1):
                                    if x2 < L and y2 < W and z2 < H:  # Positions valides
                                        solver.add_clause([
                                            -var_index(i1, v, x1, y1, z1),
                                            -var_index(i2, v, x2, y2, z2)
                                        ])

    # Résolution
    if solver.solve():
        model = solver.get_model()
        output = "SAT\n"
        for i, (L2, W2, H2, _) in enumerate(items):
            for v in range(max_v):
                for x in range(L):
                    for y in range(W):
                        for z in range(H):
                            if var_index(i, v, x, y, z) in model:
                                output += f"{v} {x} {y} {z} {x+L2} {y+W2} {z+H2}\n"
                                break
        return output.strip()
    else:
        return "UNSAT"

# Exemple d'utilisation
vehicle_dims = (40, 40, 20)
items = [
    (40, 20, 10, -1),
    (40, 20, 10, -1),
    (10, 40, 10, -1),
    (30, 40, 10, -1)
]

print(optimize_packing(vehicle_dims, items))

KeyboardInterrupt: 